# Getting Specific Answers from AI with Ollama! 🎯

Welcome to the **Structured Output** tutorial! 

Sometimes we want the AI to give us very specific types of answers, like:
- Just "Yes" or "No"
- True or False
- Multiple choice (A, B, C, or D)
- Structured data like contact information

This is called **"structured output"** because we're telling the AI exactly how to structure its answer. Let's see how to do this!

## Why is this useful?
- 🎯 **Predictable answers**: You know exactly what format you'll get back
- 🤖 **Easy to process**: Your code can reliably work with the AI's responses
- 📊 **Perfect for automation**: Build apps that depend on consistent AI responses

Let's dive in!


## Setup and Imports

First, let's import everything we need. Make sure you have Ollama running and the required packages installed!


In [11]:
# Essential imports for structured output
from pydantic import BaseModel
from typing import Literal
from ollama import chat
from datetime import date, time
import psutil, socket, subprocess, os


In [ ]:
def is_port_in_use(port: int = 11434) -> bool:
    """Check if the Ollama port is already in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def is_ollama_running() -> bool:
    """Check if Ollama process is running"""
    for proc in psutil.process_iter(['name']):
        try:
            if proc.info['name'] == 'ollama':
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False
def ensure_ollama_server():
    """
    Ensures that an Ollama server is running.
    Returns the process object if a new server was started, None if server was already running.
    """
    # First check if the server is already running
    if is_port_in_use() or is_ollama_running():
        print("✅ Ollama server is already running!")
        return None
    
    try:
        # Start the Ollama server
        print("🚀 Starting Ollama server...")
        proc = subprocess.Popen(
            ["ollama", "serve"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            start_new_session=True
        )
        
        # Wait for the server to start (max 10 seconds)
        for _ in range(10):
            if is_port_in_use():
                print(f"✅ Ollama server started successfully (PID: {proc.pid})")
                return proc
            time.sleep(1)
            
        raise TimeoutError("Server didn't start within 10 seconds")
        
    except Exception as e:
        print(f"❌ Error starting Ollama server: {e}")
        if 'proc' in locals():
            proc.terminate()
        raise

In [ ]:
# Start the server if it's not already running
ollama_process = ensure_ollama_server()

In [ ]:
!ollama ls

In [ ]:
print("✅ All imports successful! Ready to start structured output magic!")

## Simple Examples of Structured Text Output

Let's start with the basics. We'll create **Pydantic models** that tell the AI exactly what kind of answer we want.


In [2]:
# These classes tell the AI exactly what kind of answer we want
class YesNoAnswer(BaseModel):
    """For questions that should be answered with Yes or No"""
    answer: Literal["Yes", "No"]

class TrueFalseAnswer(BaseModel):
    """For statements that should be marked True or False"""
    answer: bool

class MultipleChoiceAnswer(BaseModel):
    """For questions with A, B, C, or D options"""
    answer: Literal["A", "B", "C", "D"]

print("📋 Schema classes defined! These tell the AI the exact format we want.")


📋 Schema classes defined! These tell the AI the exact format we want.


## The Magic Helper Function

Now let's create a helper function that makes getting structured answers super easy!


In [3]:
def get_structured_answer(prompt: str, answer_type: BaseModel, model: str = "qwen3:0.6b"):
    """
    Get a specific type of answer from the AI
    
    Args:
        prompt: Your question
        answer_type: What kind of answer you want (YesNoAnswer, TrueFalseAnswer, or MultipleChoiceAnswer)
        model: Which AI model to use
    """
    try:
        # Ask the AI with structured format
        response = chat(
            model=model,
            options={"temperature": 0.0},  # We want consistent answers here
            messages=[
                {"role": "user", "content": prompt}
            ],
            format=answer_type.model_json_schema()  # This is the magic! 🎩✨
        )
        
        # Get and validate the answer 
        validated_answer = answer_type.model_validate_json(response.message.content)
        return validated_answer
        
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

print("🛠️ Helper function ready! Now we can easily get structured answers.")


🛠️ Helper function ready! Now we can easily get structured answers.


## Let's Try Some Examples!

Time to see our structured output in action! 🚀


In [4]:
# Yes/No question
yes_no_q = "Is Python a programming language?"
yes_no_answer = get_structured_answer(yes_no_q, YesNoAnswer)
print(f"\n❓ Question: {yes_no_q}")
print(f"✅ Answer: {yes_no_answer.answer if hasattr(yes_no_answer, 'answer') else yes_no_answer}")



❓ Question: Is Python a programming language?
✅ Answer: Yes


In [6]:
# True/False statement
true_false_q = "The Earth is round."
true_false_answer = get_structured_answer(true_false_q, TrueFalseAnswer)
print(f"\n📝 Statement: {true_false_q}")
print(f"✅ Answer: {true_false_answer.answer if hasattr(true_false_answer, 'answer') else true_false_answer}")



📝 Statement: The Earth is round.
✅ Answer: True


In [7]:
# Multiple choice question
multiple_choice_q = """
What is the main purpose of a computer's CPU?
A) Store data permanently
B) Process instructions and perform calculations
C) Display images on the screen
D) Connect to the internet
"""
mc_answer = get_structured_answer(multiple_choice_q, MultipleChoiceAnswer)
print(f"\n🎯 Question: {multiple_choice_q}")
print(f"✅ Answer: {mc_answer.answer if hasattr(mc_answer, 'answer') else mc_answer}")



🎯 Question: 
What is the main purpose of a computer's CPU?
A) Store data permanently
B) Process instructions and perform calculations
C) Display images on the screen
D) Connect to the internet

✅ Answer: B


## More Complex Example: Friend List 👥

Now let's try something more sophisticated! We'll ask the AI to extract information about multiple people and structure it as a list.


In [8]:
# Define complex nested schemas
class FriendInfo(BaseModel):
    name: str
    age: int
    is_available: bool

class FriendList(BaseModel):
    friends: list[FriendInfo]

print("👥 Complex schema defined! This can handle lists of structured data.")


👥 Complex schema defined! This can handle lists of structured data.


In [9]:
# Let's use the complex schema
response = chat(
    model='qwen3:0.6b',
    messages=[{
        'role': 'user', 
        'content': 'I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format'
    }],
    format=FriendList.model_json_schema(),
    options={'temperature': 0},
)

# Use Pydantic to validate the response
friends_response = FriendList.model_validate_json(response.message.content)
print("\n👥 Friends extracted:")
for friend in friends_response.friends:
    status = "available" if friend.is_available else "busy"
    print(f"  • {friend.name} (age {friend.age}) - {status}")



👥 Friends extracted:
  • Ollama (age 22) - available
  • Alonso (age 23) - busy


## 🏋️‍♀️ Time for Exercises!

Now it's your turn! Complete these 5 exercises to master structured output. Follow the comments and fill in the missing code.


### Exercise 1: Star Rating ⭐

Create a rating system that only accepts 1-5 stars.


In [ ]:
# TODO: Define a StarRating model that only accepts ratings from 1 to 5
class StarRating(BaseModel):
    """Rate something on a scale from 1 to 5"""
    # YOUR CODE HERE: Add a rating field that only accepts 1, 2, 3, 4, or 5
    pass

# TODO: Use get_structured_answer to ask: "On a scale from 1 to 5, how interesting is machine learning?"
# YOUR CODE HERE

# TODO: Print the rating
# YOUR CODE HERE


### Exercise 2: Top Animals List 🐾

Get exactly three favorite animals from the AI.


In [ ]:
# TODO: Define a TopAnimals model for exactly three animals
class TopAnimals(BaseModel):
    """A list of exactly three favorite animals"""
    # YOUR CODE HERE: Create a field for exactly 3 animal names
    pass

# TODO: Ask the AI "Name your top three animals" and get structured output
# YOUR CODE HERE

# TODO: Print each animal
# YOUR CODE HERE


### Exercise 3: Contact Info Extraction 📞

Extract structured contact information from messy text.


In [ ]:
# TODO: Define a ContactInfo model with name, email, and phone fields
class ContactInfo(BaseModel):
    # YOUR CODE HERE: Add fields for name, email, and phone (all strings)
    pass

# TODO: Extract contact info from this messy text:
# "Bob Lee, contact: bob.lee@mail.com, tel: 987-6543"
messy_text = "Bob Lee, contact: bob.lee@mail.com, tel: 987-6543"
prompt = f"Extract the contact info from: '{messy_text}'"

# YOUR CODE HERE: Use get_structured_answer

# TODO: Print name, email, and phone separately
# YOUR CODE HERE


### Exercise 4: Math Quiz with Scoring 🧮

Create a quiz that returns both the answer and a score.


In [ ]:
# TODO: Define a MathQuiz model with answer (A, B, C, D) and score (integer) fields
class MathQuiz(BaseModel):
    # YOUR CODE HERE: Add answer field (A, B, C, D) and score field (int)
    pass

# TODO: Ask this math question and request both answer and score:
math_question = """
What is 2 + 2?
A) 3
B) 4
C) 5
D) 6

Return your answer choice and give yourself a score of 100 if correct, 0 if wrong.
"""

# YOUR CODE HERE: Get structured answer

# TODO: Print both the answer choice and the score
# YOUR CODE HERE


### Exercise 5: Task Scheduler 📅

Create a task with due date, priority, and description.


In [ ]:
# TODO: Define a Task model with due_date, priority, and description fields
class Task(BaseModel):
    # YOUR CODE HERE: 
    # - due_date: date type
    # - priority: Literal["Low", "Medium", "High"]
    # - description: str
    pass

# TODO: Ask the AI to create a task from this prompt:
task_prompt = "A task due on July 7, 2025 with high priority: finish the research outline."

# YOUR CODE HERE: Get structured answer

# TODO: Print each field (due_date, priority, description)
# YOUR CODE HERE


## 🎯 Tips and Best Practices

Congratulations on completing the exercises! Here are some pro tips for using structured output effectively:

### 1. 🌡️ Always Use Temperature 0 for Consistency
```python
options={"temperature": 0.0}  # Deterministic responses
```

### 2. ✅ Always Validate Responses
Use Pydantic's validation to catch mistakes:
```python
try:
    validated = MyModel.model_validate_json(response.message.content)
except ValidationError as e:
    print(f"Validation failed: {e}")
```

### 3. 🎭 Be Specific in Your Prompts
- Clear questions get better structured responses
- Include format instructions when needed
- Test your prompts with different phrasings

### 4. 🏗️ Start Simple, Then Expand
- Begin with basic schemas (Yes/No, True/False)
- Gradually add more complex nested structures
- Test each level of complexity

### 5. 🔄 Handle Errors Gracefully
Sometimes the AI might not follow the schema perfectly. Always have error handling!

## 🚀 What's Next?

Now that you've mastered structured output, you can:
- Build chatbots with predictable responses
- Create data extraction tools
- Build automated content analysis systems
- Combine structured output with other AI techniques

**Happy coding! 🎉**
